In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target

Cloning into 'Recommender-Systems-Challenge-2022'...
remote: Enumerating objects: 619, done.
remote: Counting objects: 100% (215/215), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 619 (delta 107), reused 139 (delta 52), pack-reused 404
Receiving objects: 100% (619/619), 106.18 MiB | 22.24 MiB/s, done.
Resolving deltas: 100% (279/279), done.
Updating files: 100% (254/254), done.


In [2]:
URM_all = load_urm()

#URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
#URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.85)
URM_train = sps.load_npz('Dataset/Split/URM_train.npz')
URM_test =  sps.load_npz('Dataset/Split/URM_test.npz')
URM_validation = sps.load_npz('Dataset/Split/URM_validation.npz')

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1541 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 958 ( 2.3%) Users that have less than 1 test interactions


In [3]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_train)
recommender_SLIMElasticNet.fit(topK=585, l1_ratio=0.009929794168572358, alpha=0.0018687691383859428)

SLIMElasticNetRecommender: Processed 3850 (15.7%) in 5.00 min. Items per second: 12.83
SLIMElasticNetRecommender: Processed 8046 (32.8%) in 10.00 min. Items per second: 13.41
SLIMElasticNetRecommender: Processed 12392 (50.6%) in 15.00 min. Items per second: 13.77
SLIMElasticNetRecommender: Processed 16783 (68.5%) in 20.00 min. Items per second: 13.98
SLIMElasticNetRecommender: Processed 21086 (86.0%) in 25.00 min. Items per second: 14.06
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 28.91 min. Items per second: 14.13


In [4]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

recommender_EaseR = EASE_R_Recommender(URM_train)
recommender_EaseR.fit(topK=None, normalize_matrix=False, l2_norm=225.7403645195658)

EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 10.55 min


In [5]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

recommender_RP3beta = RP3betaRecommender(URM_train)
recommender_RP3beta.fit(alpha=0.6949339074999242, beta=0.2853519331891143, topK=59, implicit=True)

RP3betaRecommender: Similarity column 24507 (100.0%), 1107.24 column/sec. Elapsed time 22.13 sec


In [6]:
print("MAP of the starting models")

result_df, _ = evaluator_validation.evaluateRecommender(recommender_SLIMElasticNet)
print("SLIM ElasticNet - MAP: {}".format(result_df.loc[10]["MAP"]))

result_df, _ = evaluator_validation.evaluateRecommender(recommender_RP3beta)
print("RP3beta - MAP: {}".format(result_df.loc[10]["MAP"]))

result_df, _ = evaluator_validation.evaluateRecommender(recommender_EaseR)
print("EaseR - MAP: {}".format(result_df.loc[10]["MAP"]))

MAP of the starting models
EvaluatorHoldout: Processed 40088 (100.0%) in 50.81 sec. Users per second: 789
SLIM ElasticNet - MAP: 0.01955257263385604
EvaluatorHoldout: Processed 40088 (100.0%) in 30.09 sec. Users per second: 1332
RP3beta - MAP: 0.01869520586455795
EvaluatorHoldout: Processed 40088 (100.0%) in 48.44 sec. Users per second: 828
EaseR - MAP: 0.019171919990306683


In [7]:
from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of three predictions scores
    R = R1*alpha + R2*beta + R3*(1-alpha-beta)
    
    Class from Dacrema exercise modified by Antonio Ercolani
    The original took as input 2 recommender

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        
        
        
    def fit(self, norm, alpha = 0.5, beta = 0.5):

        self.alpha = alpha
        self.beta = beta
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        norm_item_weights_3 = LA.norm(item_weights_3, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
            
        if norm_item_weights_3 == 0:
            raise ValueError("Norm {} of item weights for recommender 3 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * self.beta + item_weights_3 / norm_item_weights_3 * (1-self.alpha-self.beta)

        return item_weights

In [10]:
recommender_object = DifferentLossScoresHybridRecommender(URM_train, recommender_SLIMElasticNet, recommender_RP3beta, recommender_EaseR)

best_model = {
    "MAP" : 0,
    "alpha" : 0,
    "beta" : 0,
    "norm" : 0
}

norm = 1
for alpha in np.arange(0.0, 1.1, 0.1):
    for beta in np.arange(0.0, 1.1, 0.1):
            
        #truncate digits since np.arange sometimes doesn't
        alpha = round(alpha,1)
        beta = round(beta,1)
            
            
        #discard cases in which the sum is greater than 1 
        if ( (alpha+beta) <= 1): 
            theta = round(1-alpha-beta,1)
            
            print("----")
            recommender_object.fit(norm, alpha, beta)
            result_df, _ = evaluator_validation.evaluateRecommender(recommender_object)
            print("Norm: {}, Alpha: {}, Beta: {}, Theta: {}, Result: {}".format(norm, alpha, beta, 1-alpha-beta, result_df.loc[10]["MAP"]))

            if result_df.loc[10]["MAP"] > best_model["MAP"]:
                best_model["MAP"] = result_df.loc[10]["MAP"]
                best_model["alpha"] = alpha
                best_model["beta"] = beta
                best_model["norm"] = norm

                print("*** New best model found! ")
                print("New best model has MAP: {} with alpha: {}, beta: {}, theta: {}, norm: {}".format(best_model["MAP"], best_model["alpha"], best_model["beta"],
                                                                                                        1-best_model["alpha"]-best_model["beta"], best_model["norm"]))
print("----")
print("Best model has MAP: {} with alpha: {}, beta: {}, norm: {}".format(best_model["MAP"], best_model["alpha"], best_model["beta"], best_model["norm"]))

----
EvaluatorHoldout: Processed 40088 (100.0%) in 1.47 min. Users per second: 456
Norm: 1, Alpha: 0.0, Beta: 0.0, Theta: 1.0, Result: 0.019171919990306683
*** New best model found! 
New best model has MAP: 0.019171919990306683 with alpha: 0.0, beta: 0.0, theta: 1.0, norm: 1
----
EvaluatorHoldout: Processed 40088 (100.0%) in 1.46 min. Users per second: 457
Norm: 1, Alpha: 0.0, Beta: 0.1, Theta: 0.9, Result: 0.01944167078459099
*** New best model found! 
New best model has MAP: 0.01944167078459099 with alpha: 0.0, beta: 0.1, theta: 0.9, norm: 1
----
EvaluatorHoldout: Processed 40088 (100.0%) in 1.46 min. Users per second: 458
Norm: 1, Alpha: 0.0, Beta: 0.2, Theta: 0.8, Result: 0.019546698651854422
*** New best model found! 
New best model has MAP: 0.019546698651854422 with alpha: 0.0, beta: 0.2, theta: 0.8, norm: 1
----
EvaluatorHoldout: Processed 40088 (100.0%) in 1.48 min. Users per second: 452
Norm: 1, Alpha: 0.0, Beta: 0.3, Theta: 0.7, Result: 0.01959932889706097
*** New best model 